### Что используем:

In [31]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, average_precision_score, ndcg_score
from collections import Counter

### Вывод подробной информации о датасете

In [32]:

# Загрузка данных
train_df = pd.read_csv('train/train_df.csv')
test_df = pd.read_csv('test/test_df.csv')

# Общая информация о датасете
print("Train dataset info:")
print(train_df.info())
print("\nTest dataset info:")
print(test_df.info())

# Проверка наличия пропущенных значений
print("\nMissing values in train dataset:")
print(train_df.isnull().sum())
print("\nMissing values in test dataset:")
print(test_df.isnull().sum())

# Статистика по числовым признакам
print("\nStatistics for numerical features in train dataset:")
print(train_df.describe())

# Распределение целевой переменной
print("\nDistribution of target variable in train dataset:")
print(train_df['target'].value_counts(normalize=True))

# Распределение значений по search_id
print("\nNumber of unique search_id in train dataset:")
print(train_df['search_id'].nunique())
print("\nNumber of unique search_id in test dataset:")
print(test_df['search_id'].nunique())

# Размеры групп по search_id
print("\nGroup sizes by search_id in train dataset:")
print(train_df.groupby('search_id').size().describe())
print("\nGroup sizes by search_id in test dataset:")
print(test_df.groupby('search_id').size().describe())


Train dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15081 entries, 0 to 15080
Data columns (total 81 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   search_id   15081 non-null  int64  
 1   feature_0   15081 non-null  int64  
 2   feature_1   15081 non-null  int64  
 3   feature_2   15081 non-null  int64  
 4   feature_3   15081 non-null  int64  
 5   feature_4   15081 non-null  int64  
 6   feature_5   15081 non-null  int64  
 7   feature_6   15081 non-null  int64  
 8   feature_7   15081 non-null  int64  
 9   feature_8   15081 non-null  int64  
 10  feature_9   15081 non-null  int64  
 11  feature_10  15081 non-null  int64  
 12  feature_11  15081 non-null  int64  
 13  feature_12  15081 non-null  int64  
 14  feature_13  15081 non-null  int64  
 15  feature_14  15081 non-null  int64  
 16  feature_15  15081 non-null  int64  
 17  feature_16  15081 non-null  float64
 18  feature_17  15081 non-null  float64
 19  featu

### Подготовка данных

In [33]:
# Подготовка данных
train_df = pd.read_csv('train/train_df.csv')
test_df = pd.read_csv('test/test_df.csv')

X_train = train_df.drop(['target'], axis=1)
y_train = train_df['target']
groups_train = train_df.groupby('search_id').size().to_numpy()

X_test = test_df.drop(['target'], axis=1)
y_test = test_df['target']
groups_test = test_df.groupby('search_id').size().to_numpy()

### Пересемплирование

#### значение target[0]=0.978649 и target[1]=0.021351, что говорит о несбалансированности датасета


In [34]:
# Пересемплирование (будем пересемплировать без удаления 'search_id' из X_train)
ros = RandomOverSampler(random_state=0)
# Преобразование для сохранения search_id
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
# Обновление размеров групп после пересемплирования
search_ids_resampled = X_resampled['search_id']
groups_resampled = X_resampled.groupby('search_id').size().to_numpy()

# Подготовка данных для обучения и тестирования
X_train_resampled = X_resampled.drop(['search_id'], axis=1)
X_test_prepared = X_test.drop(['search_id'], axis=1)

### Обучение модели

In [35]:
# Обучение модели LGBMRanker
gbm = lgb.LGBMRanker(objective='lambdarank', metric='ndcg', learning_rate=0.1, num_leaves=31, verbose=1)
gbm.fit(X_train_resampled, y_resampled, group=groups_resampled, eval_set=[(X_test_prepared, y_test)], eval_group=[groups_test],
        eval_at=[5, 20, 40], callbacks=[lgb.early_stopping(stopping_rounds=10)])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009553 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12389
[LightGBM] [Info] Number of data points in the train set: 29518, number of used features: 75
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's ndcg@5: 0.872973	valid_0's ndcg@20: 0.902789	valid_0's ndcg@40: 0.902789


LGBMRanker(metric='ndcg', objective='lambdarank', verbose=1)

In [37]:
# Получение предсказаний для тестового набора данных
y_pred = gbm.predict(X_test_prepared)


# Группировка истинных меток и предсказаний по search_id
grouped_true = test_df.groupby('search_id')['target'].apply(list)
grouped_scores = test_df.assign(pred=y_pred).groupby('search_id')['pred'].apply(list)

print("len grouped_scores: ", len(grouped_scores))


print("len grouped_true: ", len(grouped_true))

count = 0

ndcg_values = []
for search_id in grouped_true.index:
    if len(set(grouped_true[search_id])) > 1:  # Есть ли вариативность в релевантности
        true_relevance = np.asarray([grouped_true[search_id]])
        scores = np.asarray([grouped_scores[search_id]])
        ndcg_values.append(ndcg_score(true_relevance, scores))
    # else:
        # print(f"Skipping NDCG calculation for search_id {search_id} due to lack of relevance variation.")
        
if ndcg_values:  # Проверяем, что список не пустой
    average_ndcg = np.mean(ndcg_values)
    print(f'Среднее NDCG по всем группам search_id: {average_ndcg}')




len grouped_scores:  100
len grouped_true:  100
Среднее NDCG по всем группам search_id: 0.5915904306795731


Исходя из:
```
Среднее NDCG по всем группам search_id: 0.5915904306795731
```

и процесса обучения:

```
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009338 seconds.
[LightGBM] [Info] Total Bins 12389
[LightGBM] [Info] Number of data points in the train set: 29518, number of used features: 75
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's ndcg@5: 0.872973	valid_0's ndcg@20: 0.902789	valid_0's ndcg@40: 0.902789
```

Можно сделать следующие выводы:

### Выводы:
1. **Качество модели**: Модель демонстрирует хорошее начальное качество ранжирования согласно метрикам NDCG@5, NDCG@20 и NDCG@40 на валидационном наборе данных. Это указывает на то, что модель эффективно выделяет наиболее релевантные объекты на верхние позиции списка.

2. **Среднее значение NDCG**: Среднее значение NDCG по всем группам `search_id` составляет 0.5915904306795731 после исключения групп без вариативности релевантности. Это говорит о том, что в среднем модель хорошо справляется с задачей ранжирования, учитывая разнообразие и сложность запросов в данных.

### Что можно добавить:
1. **Тюнинг модели**: Учитывая, что модель достигла условия остановки уже после первого раунда, нужно рассмотреть возможность адаптации параметров обучения, включая `learning_rate`, `num_leaves` и `early_stopping_rounds`, чтобы попытаться улучшить качество модели. Возможно, стоит увеличить количество раундов для `early_stopping` или отрегулировать скорость обучения.

2. **Анализ признаков**: Поскольку использовалось 75 признаков, стоит провести анализ значимости признаков, чтобы определить, все ли из них вносят вклад в качество ранжирования. Удаление менее важных признаков может помочь упростить модель и сосредоточить внимание на наиболее информативных данных.

3. **Эксперименты с размером групп**: Поскольку расчет NDCG был скорректирован для групп с вариативностью релевантности, полезно проанализировать, как размер группы влияет на качество ранжирования. Эксперименты с различными стратегиями группировки и пересемплирования могут выявить оптимальные подходы к обучению модели.

В целом, исходя из результатов, видно что модель `LGBMRanker` обладает хорошим потенциалом для решения задачи ранжирования в данном наборе данных. Дальнейшая оптимизация и тонкая настройка могут помочь улучшить эти результаты еще больше.